# 2021/05/13

$\newcommand{\ket}[1]{| #1 \rangle}$
$\newcommand{\bra}[1]{\angle #1 |}$

## グローバーのアルゴリズムの実装

まず必要な環境をセットアップします。

In [ ]:
# Tested with python 3.7.9, qiskit 0.23.5, numpy 1.20.1
import matplotlib.pyplot as plt
import numpy as np

# Qiskit関連のパッケージをインポート
from qiskit import IBMQ, Aer, QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor

### グローバーのアルゴリズムを実装する回路

6量子ビットの回路`grover_circuit`を準備します。

In [ ]:
n = 6
grover_circuit = QuantumCircuit(n)

グローバー反復を一回実行する量子回路は以下のような構造になりますが、赤枠で囲んだ部分（オラクル$U_w$とDiffuserの中の$U_0=2\ket{0}\bra{0}-I$の部分）を実装する量子回路を書いてみましょう。

```{image} figs/grover_6bits_45.png
:alt: grover_6bits_45
:class: bg-primary mb-1
:width: 600px
:align: center
```

一様な重ね合わせ状態$\ket{s}$を生成した後に、オラクルを実装します。

In [ ]:
def initialize_s(qc, qubits):
    """回路のqubitsにHゲートを適用"""
    for q in qubits:
        qc.h(q)
    return qc

grover_circuit = initialize_s(grover_circuit, list(range(n)))

# オラクルUwを作成して、回路に実装
oracle = QuantumCircuit(n)

##################
### EDIT BELOW ###
##################

# Oracle Uw?

##################
### EDIT ABOVE ###
##################

oracle_gate = oracle.to_gate()
oracle_gate.name = "U_w"

grover_circuit.append(oracle_gate, list(range(n)))

**解答例**

```python
oracle.x(1)
oracle.x(4)
oracle.h(n-1)
oracle.mct(list(range(n-1)), n-1)
oracle.h(n-1)
oracle.x(1)
oracle.x(4)
```

次に、Diffuser用の回路を実装します。

In [ ]:
def diffuser(n)
    qc = QuantumCircuit(n)

    qc.h(range(n))

    ##################
    ### EDIT BELOW ###
    ##################

    # Unitary U0?

    ##################
    ### EDIT ABOVE ###
    ##################
    
    qc.h(range(n))

    U_s = qc.to_gate()
    U_s.name = "U_s"
    return U_s

**解答例**

```python
    #
    qc.rz(2*np.pi, n-1)
    qc.x(list(range(n)))

    # Multi-controlled Zゲート
    qc.h(n-1)
    qc.mct(list(range(n-1)), n-1)
    qc.h(n-1)

    qc.x(list(range(n)))
```

回路ができたので、printして確認してみます。

In [ ]:
grover_circuit.append(diffuser(n), list(range(n)))
grover_circuit.measure_all()
grover_circuit.draw('mpl')

### シミュレータでの実験

回路の実装ができたら、シミュレータで実行して結果をプロットしてみます。結果が分かりやすくなるように、測定したビット列を整数にしてからプロットするようにしてみます。

In [ ]:
backend = Aer.get_backend('qasm_simulator')
results = execute(grover_circuit, backend=backend, shots=1024).result()
answer = results.get_counts()

# 横軸を整数でプロットする
def show_distribution(answer):
    n = len(answer)
    x = [int(key,2) for key in list(answer.keys())]
    y = list(answer.values())

    fig, ax = plt.subplots()
    rect = ax.bar(x,y)

    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{:.3f}'.format(height/sum(y)),
                        xy=(rect.get_x()+rect.get_width()/2, height),xytext=(0,0),
                        textcoords="offset points",ha='center', va='bottom')
    autolabel(rect)
    plt.ylabel('Probabilities')
    plt.show()

show_distribution(answer)

正しく回路が実装できていれば、$\ket{101101}=\ket{45}$の状態を高い確率で測定できる様子を見ることができるでしょう。

### 量子コンピュータでの実験

では次に、この量子回路を実機の量子コンピュータで実行して、結果を確認してみましょう。

#### 使う量子コンピュータの選定

In [1]:
# IBM量子コンピュータで実行する場合
IBMQ.enable_account('__paste_your_token_here__')

# 使うIBMQバックエンドの選択
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 6 and
                                   not x.configuration().simulator and x.status().operational==True))
print("least busy backend: ", backend)

NameError: name 'IBMQ' is not defined

#### 実行

In [ ]:
# 最も空いているバックエンドで回路を実行します。キュー内のジョブの実行をモニターします。
job = execute(grover_circuit, backend=backend, shots=1024, optimization_level=3)
job_monitor(job, interval=2)

#### 結果の確認

In [ ]:
# 計算結果
results = job.result()
answer = results.get_counts(grover_circuit)
show_distribution(answer)

In [ ]:
# (Hidden cell) set to some dummy dict
answer = {'000000': 21, '000001': 15, '010000': 21, '010001': 10, '010010': 18, '010011': 14, '010100': 22, '010101': 13, '010110': 21, '010111': 11, '011000': 16, '011001': 9, '011010': 15, '011011': 12, '011100': 20, '011101': 13, '011110': 19, '011111': 11, '000010': 14, '100000': 26, '100001': 23, '100010': 20, '100011': 11, '100100': 16, '100101': 12, '100110': 13, '100111': 15, '101000': 19, '101001': 17, '101010': 13, '101011': 14, '101100': 17, '101101': 18, '101110': 23, '101111': 9, '000011': 21, '110000': 19, '110001': 17, '110010': 9, '110011': 16, '110100': 23, '110101': 21, '110110': 13, '110111': 8, '111000': 14, '111001': 20, '111010': 12, '111011': 9, '111100': 13, '111101': 17, '111110': 11, '111111': 8, '000100': 17, '000101': 18, '000110': 24, '000111': 19, '001000': 13, '001001': 15, '001010': 20, '001011': 16, '001100': 20, '001101': 13, '001110': 19, '001111': 18}
show_distribution(answer)

シミュレータに比べると結果は非常に悪いですね。。。残念ながら、今の量子コンピュータをそのまま使うとこういう結果になってしまいます。{ref}`エラー緩和 <measurement_error_mitigation>`等のテクニックを使うことである程度改善することはできますが、それは今後の課題としましょう。